In [78]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
from scipy.optimize import minimize

In [2]:
import multiprocessing as mp
from itertools import product

# Simplest Situation

In [3]:
def reflect_line(theta_c, a, R): # theta_c : cherenkov angle, a : emission point, R : mirror radius
    x0 = (a*np.tan(theta_c)**2 + np.sqrt(R**2 + (R**2-a**2)*np.tan(theta_c)**2))/(1+np.tan(theta_c)**2)
    y0 = np.tan(theta_c)*(-a+np.sqrt(R**2 + (R**2-a**2)*np.tan(theta_c)**2))/(1+np.tan(theta_c)**2)
    tanTheta = y0/x0
    sin2Theta = 2*tanTheta/(1+tanTheta**2)
    cos2Theta = (1-tanTheta**2)/(1+tanTheta**2)
    A = (sin2Theta-np.tan(theta_c)*cos2Theta)/(cos2Theta+np.tan(theta_c)*sin2Theta)
    B = a*np.tan(theta_c)/(cos2Theta+np.tan(theta_c)*sin2Theta)
    return np.array([A, B, tanTheta])

In [4]:
R = 100.0 # Mirror Radius [cm]
a = 50 #cm
#theta_c = np.arange(-np.pi/2.0,np.pi/2.0, 0.1)
theta_c = np.arange(-1,1, 0.05)
n_Aero = 1.021
n_Air = 1.000273
Thick_Aero = 10 #cm
theta = np.arcsin(n_Aero*np.sin(theta_c)/n_Air)
b = a - Thick_Aero*np.tan(theta_c)/np.tan(theta)

line1 = reflect_line(np.arcsin(theta), a, R) # back Aerogel 
line2 = reflect_line(np.arcsin(theta), b, R) # front Aerogel

X = (-line1[1]+line2[1])/(line1[0]-line2[0])
Y = (line1[0]*line2[1]-line1[1]*line2[0])/(line1[0]-line2[0])

C:\Users\shota\AppData\Local\Temp\ipykernel_9336\2806016024.py:11: RuntimeWarning: invalid value encountered in arcsin
  line1 = reflect_line(np.arcsin(theta), a, R) # back Aerogel
C:\Users\shota\AppData\Local\Temp\ipykernel_9336\2806016024.py:12: RuntimeWarning: invalid value encountered in arcsin
  line2 = reflect_line(np.arcsin(theta), b, R) # front Aerogel


In [5]:
TITLE_FONTSIZE = 35
AXISTITLE_FONTSIZE = 22
LEGEND_FONTSIZE = 22
TICK_FONTSIZE = 15
MARKERSIZE = 20

layout = go.Layout(
    width=720,
    height=540,
    template="simple_white",
    title_font=dict(size=TITLE_FONTSIZE, family="Times New Roman"),
    title=r"Focal Plane R/2=" + str("{:.1f}".format(R/2)) + "cm",
    xaxis=dict(
        showgrid=True,
        mirror=True,
        range=(0., 150),
        title=r"<i>x</i> [cm]",
        title_font=dict(size=AXISTITLE_FONTSIZE, family="Times New Roman"),
        tickfont=dict(size=TICK_FONTSIZE),
        #dtick=10,
    ),
    yaxis=dict(
        showgrid=True,
        mirror=True,
        range=(-80, 80),
        title=r"<i>y</i> [cm]",
        title_font=dict(size=AXISTITLE_FONTSIZE, family="Times New Roman"),
        tickfont=dict(size=TICK_FONTSIZE),
        dtick=20,
    ),
    legend_font=dict(size=LEGEND_FONTSIZE, family="Times New Roman"),
)

In [44]:
FP1 = go.Scatter(
        x=X,
        y=Y,
        mode="lines",
        name=r"focal plane",
        line=dict(
            color="red", 
            width=2,
            #shape="hvh"
        ),
        #showlegend=True,
        showlegend=False,
    )
fig = go.Figure(FP1, layout=layout)
fig.show()

# 2D Complex Situation

In [17]:
def reflect_line2D(theta_c, EmPointX, R, x0Mirror, y0Mirror, x0beam, y0beam, theta_beam, L_em=5, fBack = True, fsin=True):
    n_Aero = 1.021 # refractive index
    n_Air = 1.000273 # refractive index
    if fBack: # Emission Point is Back of Aerogel
        b = np.tan(theta_beam)*(EmPointX-x0beam) + y0beam # y-coordinate of emission point
    elif fsin: # Emission Point is Front & L_em is flight length of light
        b = L_em*np.sin(theta_beam+theta_c) + np.tan(theta_beam)*(EmPointX-L_em*np.cos(theta_beam+theta_c)-x0beam) + y0beam
    else: # Emission Point is Front & L_em is difference of x-coordinate between front emission point and back
        b = L_em*np.tan(theta_beam+theta_c) + np.tan(theta_beam)*(EmPointX-L_em-x0beam) + y0beam
    tantheta = np.tan(theta_beam+np.arcsin(n_Aero*np.sin(theta_c)/n_Air)) # slope of emission light line
    A0 = np.sqrt(R**2-(b-y0Mirror)**2 - 2*(x0Mirror-EmPointX)*(b-y0Mirror)*tantheta + (R**2-(x0Mirror-EmPointX)**2)*tantheta**2) # part of sqrt in xM ,yM
    xM = (x0Mirror-(b-y0Mirror-EmPointX*tantheta)*tantheta +A0 )/(1+tantheta**2) # x-coordinate of intersection of Mirror & emission light line
    yM = tantheta*(x0Mirror-(b-y0Mirror)*tantheta - EmPointX + A0 )/(1+tantheta**2) + b # y-coordinate of intersection of Mirror & emission light line

    tanTheta = (yM-y0Mirror)/(xM-x0Mirror) # slope of Perpendicular of Mirror at (xM, yM)
    sin2Theta = 2*tanTheta/(1+tanTheta**2)
    cos2Theta = (1-tanTheta**2)/(1+tanTheta**2)

    A = (sin2Theta-tantheta*cos2Theta)/(cos2Theta+tantheta*sin2Theta)
    B = (EmPointX*tantheta-b + y0Mirror-x0Mirror*tantheta - (x0Mirror*sin2Theta-y0Mirror*cos2Theta) + (x0Mirror*cos2Theta+y0Mirror*sin2Theta)*tantheta)/(cos2Theta+tantheta*sin2Theta)

    return [A,B, tanTheta, xM, yM]


In [18]:
R = 100.0 # Mirror Radius [cm]
a = 60.0 # x-coordinate of Back end of Aerogel [cm] 
#theta_c = np.arange(-0.201,0.201, 0.01) # Cherenkov Angle
theta_c = np.arange(-1.001,1.001, 0.01) # Cherenkov Angle
Thick_Aero = 10 #cm
L_em = 10 #cm 
theta_beam = 0.015 # rad
x0beam = -100.0 #cm
y0beam = 1.0 #cm
#theta_mirror = 20.0 # degree
#x0Mirror = R - R*np.cos(np.deg2rad(theta_mirror))
#y0Mirror = R*np.sin(np.deg2rad(theta_mirror))
theta_mirror = 0.17*2 # rad
#x0Mirror = R - R*np.cos(theta_mirror)
#y0Mirror = R*np.sin(theta_mirror)
x0Mirror = 13
y0Mirror = 21

#theta = np.arcsin(n_Aero*np.sin(theta_c)/n_Air)
#b = a - Thick_Aero*np.tan(theta_c)/np.tan(theta)

line3 = reflect_line2D(theta_c, a, R, x0Mirror, y0Mirror, x0beam, y0beam, theta_beam)
line4 = reflect_line2D(theta_c, a, R, x0Mirror, y0Mirror, x0beam, y0beam, theta_beam, L_em=L_em, fBack=False, fsin=False)

X2 = (-line3[1]+line4[1])/(line3[0]-line4[0])
Y2 = (line3[0]*line4[1]-line3[1]*line4[0])/(line3[0]-line4[0])

In [19]:
TITLE_FONTSIZE = 35
AXISTITLE_FONTSIZE = 22
LEGEND_FONTSIZE = 22
TICK_FONTSIZE = 15
MARKERSIZE = 20

layout2 = go.Layout(
    width=720,
    height=720,
    template="simple_white",
    title_font=dict(size=TITLE_FONTSIZE, family="Times New Roman"),
    title=r"Focal Plane (R/2=" + str("{:.1f}".format(R/2)) + "cm)",
    xaxis=dict(
        showgrid=True,
        mirror=True,
        range=(0., 120),
        #range=(-2., 2),
        title=r"<i>x</i> [cm]",
        title_font=dict(size=AXISTITLE_FONTSIZE, family="Times New Roman"),
        tickfont=dict(size=TICK_FONTSIZE),
        dtick=10,
    ),
    yaxis=dict(
        showgrid=True,
        mirror=True,
        range=(-80, 80),
        title=r"<i>y</i> [cm]",
        title_font=dict(size=AXISTITLE_FONTSIZE, family="Times New Roman"),
        tickfont=dict(size=TICK_FONTSIZE),
        scaleanchor="x", 
        scaleratio=1,
        dtick=10,
    ),
    legend_font=dict(size=LEGEND_FONTSIZE, family="Times New Roman"),
)

In [20]:
FP2 = go.Scatter(
        x=X2,
        y=Y2,
        #x=line3[3],
        #y=line3[4],
        #x=theta,
        #y=np.sqrt((line3[3]-x0Mirror)**2 + (line3[4]-y0Mirror)**2),
        mode="lines",
        name=r"focal plane",
        line=dict(
            color="red", 
            width=2,
            #shape="hvh"
        ),
        #showlegend=True,
        showlegend=False,
    )
fig2 = go.Figure(FP2, layout=layout2)
fig2.show()

# 3D ray tracing

## 3次元の回転行列

In [3]:
def RotMatrix(xM, yM, zM, x0Mirror, y0Mirror, z0Mirror, Radius):
    r11 = ( (xM - x0Mirror)**2 - (yM - y0Mirror)**2 - (zM - z0Mirror)**2 )/ Radius**2
    r22 = ( -(xM - x0Mirror)**2 + (yM - y0Mirror)**2 - (zM - z0Mirror)**2 )/ Radius**2
    r33 = ( - (xM - x0Mirror)**2 - (yM - y0Mirror)**2 + (zM - z0Mirror)**2 )/ Radius**2
    r12 = 2*(xM-x0Mirror)*(yM-y0Mirror)/Radius**2
    r21 = r12
    r13 = 2*(xM-x0Mirror)*(zM-z0Mirror)/Radius**2
    r31 = r13
    r23 = 2*(yM-y0Mirror)*(zM-z0Mirror)/Radius**2
    r32 = r23
    
    #return np.matrix([[r11, r12, r13], [r21, r22, r23], [r31, r32, r33]])
    return np.array([[r11, r12, r13], [r21, r22, r23], [r31, r32, r33]])

In [4]:
RotMatrix(10, 20, 30, 0,0,0, 100)

array([[-0.12,  0.04,  0.06],
       [ 0.04, -0.06,  0.12],
       [ 0.06,  0.12,  0.04]])

## 球面鏡で反射された光の方向ベクトルを求める

In [11]:
def direction_ref(xM, yM, zM, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius):
    L_e = np.sqrt((xM-xE)**2 + (yM-yE)**2 + (zM-zE)**2)
    d_emission = np.array([[(xM-xE)/L_e], [(yM-yE)/L_e], [(zM-zE)/L_e]])
    RotM = RotMatrix(xM, yM, zM, x0Mirror, y0Mirror, z0Mirror, Radius)
    return -1 * (RotM@d_emission).flatten()

In [19]:
d_ref = direction_ref(101.4849400268, 19.9809400966, 0, 5, 0, 0, 1.5191739036 ,17.3645296907, 0, 100)
d_ref

array([-0.9884902 ,  0.15128494, -0.        ])

In [17]:
np.linalg.norm(d_ref, ord=2)

1.000000000001042

In [13]:
#0.9884902/1.0094637497
-1*d_ref[0]/1.0094637497

0.9792230735063551

In [14]:
#0.15128494/0.1544948692
d_ref[1]/0.1544948692

0.979223073185668

## 反射された光と検出点との距離

In [31]:
def distance_func(xM, yM, zM, xD, yD, zD, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius):
    rD = np.array([xD-xM, yD-yM, zD-zM])
    d_ref = direction_ref(xM, yM, zM, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius)
    return np.linalg.norm(np.cross(rD, d_ref), ord=2)/np.linalg.norm(d_ref, ord=2)

In [39]:
distance_func(101.4849400268, 19.9809400966, 0, 42.4757289991, 39.5032559114, 0, 5, 0, 0, 1.5191739036 ,17.3645296907, 0, 100)

10.370412792092068

## Scipy.optimize.minimize を用いた反射点計算

### 拘束条件 (反射点が球面上にある)

In [123]:
def equality_constranint(x, *args):
    xM, yM, zM = x[0], x[1], x[2]
    x0Mirror, y0Mirror, z0Mirror, Radius = args[0], args[1], args[2], args[3]
    xD, yD, zD   = args[4], args[5], args[6], args[7]
    xE, yE, zE = args[8], args[9], args[10]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0][0], args[0][1], args[0][2], args[0][3]
#    xD, yD, zD   = args[0][4], args[0][5], args[0][6], args[0][7]
#    xE, yE, zE = args[0][8], args[0][9], args[0][10]
    return (xM-x0Mirror)**2 + (yM-y0Mirror)**2 + (zM-z0Mirror)**2 - Radius**2

In [125]:
def equality_constranint2(x, xD, yD, zD, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius):
    xM, yM, zM = x[0], x[1], x[2]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0], args[1], args[2], args[3]
#    xD, yD, zD   = args[4], args[5], args[6], args[7]
#    xE, yE, zE = args[8], args[9], args[10]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0][0], args[0][1], args[0][2], args[0][3]
#    xD, yD, zD   = args[0][4], args[0][5], args[0][6], args[0][7]
#    xE, yE, zE = args[0][8], args[0][9], args[0][10]
    return (xM-x0Mirror)**2 + (yM-y0Mirror)**2 + (zM-z0Mirror)**2 - Radius**2

### 反射点を変数として検出点と反射光との距離を求める関数

In [124]:
def distance_f(x, *args):
    xM, yM, zM = x[0], x[1], x[2]
    x0Mirror, y0Mirror, z0Mirror, Radius = args[0], args[1], args[2], args[3]
    xD, yD, zD   = args[4], args[5], args[6], args[7]
    xE, yE, zE = args[8], args[9], args[10]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0][0], args[0][1], args[0][2], args[0][3]
#    xD, yD, zD   = args[0][4], args[0][5], args[0][6], args[0][7]
#    xE, yE, zE = args[0][8], args[0][9], args[0][10]

    rD = np.array([xD-xM, yD-yM, zD-zM])
    d_ref = direction_ref(xM, yM, zM, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius)
    return np.linalg.norm(np.cross(rD, d_ref), ord=2)/np.linalg.norm(d_ref, ord=2)

In [126]:
def distance_f2(x, xD, yD, zD, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius):
    xM, yM, zM = x[0], x[1], x[2]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0], args[1], args[2], args[3]
#    xD, yD, zD   = args[4], args[5], args[6], args[7]
#    xE, yE, zE = args[8], args[9], args[10]
#    x0Mirror, y0Mirror, z0Mirror, Radius = args[0][0], args[0][1], args[0][2], args[0][3]
#    xD, yD, zD   = args[0][4], args[0][5], args[0][6], args[0][7]
#    xE, yE, zE = args[0][8], args[0][9], args[0][10]

    rD = np.array([xD-xM, yD-yM, zD-zM])
    d_ref = direction_ref(xM, yM, zM, xE, yE, zE, x0Mirror, y0Mirror, z0Mirror, Radius)
    return np.linalg.norm(np.cross(rD, d_ref), ord=2)/np.linalg.norm(d_ref, ord=2)

### 初期位置とインプットパラメータ

In [136]:
x0 = np.array([101.4849400268, 9.9072166291, 0.0])
args = (1.5191739036 ,17.3645296907, 0.0, 100.0, 52.5871498729, 27.465745449, 0.0, 5.0, 0.0, 0.0) # 球面中心(x,y,z), 半径, 検出点(x,y,z), 放出点(x,y,z) 
args2 = (52.5871498729, 27.465745449, 0.0, 5.0, 0.0, 0.0, 1.5191739036 ,17.3645296907, 0.0, 100.0) # 検出点(x,y,z), 放出点(x,y,z), 球面中心(x,y,z), 半径
args3 = (52.525161348, 6.275469636, 0.0, 5.0, 0.0, 0.0, 1.5191739036 ,17.3645296907, 0.0, 100.0) # 検出点(x,y,z), 放出点(x,y,z), 球面中心(x,y,z), 半径

### 拘束条件

In [ ]:
cons = {"type":"eq", "fun":equality_constranint} 
cons2 = {"type":"eq", "fun":equality_constranint2, "args":args2} 
cons3 = {"type":"eq", "fun":equality_constranint2, "args":args3} 

### 距離を最小化

In [137]:
#result = minimize(fun=distance_f, x0=x0, args=args, method="SLSQP", constraints=cons)
#result = minimize(fun=distance_f2, x0=x0, args=args2, method="SLSQP", constraints=cons2)
result = minimize(fun=distance_f2, x0=x0, args=args3, method="SLSQP", constraints=cons3)

In [138]:
r0Mirror = np.array([1.5191739036 ,17.3645296907, 0.0])
np.linalg.norm(result.x-r0Mirror, ord=2)

100.00000000001616

In [139]:
result

     fun: 7.218607919359754e-07
     jac: array([-0.12901893, -0.23043663, -0.49209559])
 message: 'Optimization terminated successfully'
    nfev: 103
     nit: 22
    njev: 22
  status: 0
 success: True
       x: array([ 9.48272339e+01, -1.86022042e+01, -1.25230961e-06])